# Personal functions to be called throughout the code

In [1]:
def standardize(parameter):
    """
    :param parameter: the array with the parameters you wish to standardize
    :return param_mean: mean of the input array
    :return param_std : standard deviation of the input array
    :return stdized_param: final standardized array
    """
    import numpy as np
    
    param_mean    = np.mean(parameter)
    param_std     = np.std(parameter)
    stdized_param = []                  # standardized parameter -- output
    for i in range(parameter.size):
        param_temp = (parameter[i] - param_mean)/param_std
        stdized_param.append(param_temp)
    stdized_param = np.array(stdized_param)
    return (param_mean, param_std, stdized_param)

In [54]:
def un_standardize(stdized_par, mean, std):
    """
    This function undoes what the former does!
    """
    
    import numpy as np
    
    recovered_par = []
    for i in range(stdized_par.size):
        param_temp = stdized_par[i]*std + mean
        recovered_par.append(param_temp)
    recovered_par = np.array(recovered_par)
    return (recovered_par)

In [ ]:
# def my_plots(x, row, col, position):
#     import numpy             as np
#     import matplotlib.pyplot as plt
#     import seaborn           as sns
    
# #     plot_dict = {}
# #     plot_dict["plot{0}{1}".format(row, col)] 
#     fig = plt.subplot(row+1, col+1, position)
#     sns.kdeplot(x, shade=True, c='#e6550d')
#     plt.xlabel(r"$\beta_{%d%d}$" % (row, col), fontsize=12)
#     plt.tick_params('both', labelsize='12')
#     plt.tight_layout()
    
#     return (fig)    

# Libraries

In [2]:
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import statsmodels.api   as sm
import seaborn           as sns
import pystan
import time

# Configuring the regression parameters

In [3]:
my_data = pd.read_csv('../../Catalogue/binom_reg_dataset.csv')

In [4]:
# filtering the dataset
redshifts = my_data['Z']
index     = np.where(redshifts.values<=0.4)

# parameters of interest
logit_class = my_data['LOGIT_CLASS(1-UVUP;0-UVWEAK)'].values[index]    # y axis: logit class -- uv upturn yes or no
whan_class  = my_data['WHAN(0-NA;1-RP;2-wA;3-sA;4-SF)'].values[index]  # My types of galaxies
mass        = my_data['STELLAR_MASS'].values[index]                    # 1st parameter
redshift    = redshifts.values[index]                                  # 2nd parameter

In [51]:
# original parameters
mean_mass, std_mass, x2 = standardize(mass)
x1      = redshift
y       = logit_class
classes = (whan_class+1).astype(int)        # Must sum +1 to avoid index issues with STAN
n_obs   = x1.size
n_class = np.unique(classes).size

# new parameters - important for plotting!
n_obs2 = 30
x1_sim = np.linspace(x1.min(), x1.max(), n_obs2)
x2_sim = np.linspace(x2.min(), x2.max(), n_obs2)

# grid
plot_x1, plot_x2 = np.meshgrid(x1_sim, x2_sim)  # THIS IS WHERE THE GRID IS DONE
plot_x1 = plot_x1.reshape((n_obs2**2), 1)
plot_x2 = plot_x2.reshape((n_obs2**2), 1)

In [15]:
print x2.min(), x2.max() # sanity check
print np.median(x2)
print plot_x1.shape

-3.0048533107143856 2.6904846038511474
0.018866401595684694
(900, 1)


In [16]:
# dataset to be used in the regression
regression_data      = {}                                                 # Dictionary, as stated in the pystan manual
regression_data['Y'] = y
regression_data['X'] = sm.add_constant(np.column_stack((x1, x1**2, x2, x2**2)))
regression_data['K'] = regression_data['X'][0,:].size                     # Number of betas -- b0, b1, b2, b3, b4
regression_data['W'] = classes
regression_data['N'] = n_obs
regression_data['C'] = n_class                                      # Number of different classes (partial pooling)

# dataset to be used in the plot -- after meshgrid
regression_data['X2'] = sm.add_constant(np.column_stack((plot_x1, plot_x1**2, plot_x2, plot_x2)))
regression_data['N2'] = n_obs2**2

In [17]:
print regression_data['X2'].shape

(900, 5)


In [18]:
# Fit: STAN code ----------------------------------------------------------------------------------------------------------
stan_code = """
// DECLARATION OF VARIABLES -----------------------------------------------------------------------------------------------
data{
    int<lower=1> N;
    int<lower=1> N2;
    int<lower=1> K;
    int<lower=1> C;
    int W[N];
    int<lower=0, upper=1> Y[N];
    matrix[N, K] X;         // redshift and stellar mass 
    matrix[N2,K] X2;        // redshift and stellar mass -- after grid
    }

// DEFINING THE PRIOR(S) --------------------------------------------------------------------------------------------------
parameters{
    matrix[K,C] beta;       // 25 betas!
    real<lower=0> sigma;    // Shared hyperpriors
    real mu;                // Shared hyperpriors
    }

// MODEL: PROBABILITY, HYPERPRIORS, PRIORS, AND REGRESSION ----------------------------------------------------------------
model {
   vector[N] prob;
    for (i in 1:N) {
      prob[i] = beta[1,W[i]]*X[i,1] + beta[2,W[i]]*X[i,2] + beta[3,W[i]]*X[i,3] + beta[4,W[i]]*X[i,4] + 
      beta[5,W[i]]*X[i,5];
      }

    sigma ~ gamma(0.001, 0.001);                           // shared hyperpriors
    mu ~ normal(0, 100);                                   // shared hyperpriors
     
    for (i in 1:K) {
       for (j in 1:C) beta[i,j] ~ normal(mu, sigma);       // priors
        }

    Y ~ bernoulli_logit(prob);                             // regression
    }

// DATA TO BE PLOTTED -----------------------------------------------------------------------------------------------------
generated quantities{
    vector[N2] prob01;
    vector[N2] eta01;
    vector[N2] prob02;
    vector[N2] eta02;
    vector[N2] prob03;
    vector[N2] eta03;
    vector[N2] prob04;
    vector[N2] eta04;
    vector[N2] prob05;
    vector[N2] eta05;
    
    for(j in 1:N2){
        eta01[j] = beta[1,1]*X2[j,1] + beta[2,1]*X2[j,2] + beta[3,1]*X2[j,3] + beta[4,1]*X2[j,4] + beta[5,1]*X2[j,5];
        eta02[j] = beta[1,2]*X2[j,1] + beta[2,2]*X2[j,2] + beta[3,2]*X2[j,3] + beta[4,2]*X2[j,4] + beta[5,2]*X2[j,5];
        eta03[j] = beta[1,3]*X2[j,1] + beta[2,3]*X2[j,2] + beta[3,3]*X2[j,3] + beta[4,3]*X2[j,4] + beta[5,3]*X2[j,5];
        eta04[j] = beta[1,4]*X2[j,1] + beta[2,4]*X2[j,2] + beta[3,4]*X2[j,3] + beta[4,4]*X2[j,4] + beta[5,4]*X2[j,5];
        eta05[j] = beta[1,5]*X2[j,1] + beta[2,5]*X2[j,2] + beta[3,5]*X2[j,3] + beta[4,5]*X2[j,4] + beta[5,5]*X2[j,5];
        prob01[j] = inv_logit(eta01[j]);
        prob02[j] = inv_logit(eta02[j]);
        prob03[j] = inv_logit(eta03[j]);
        prob04[j] = inv_logit(eta04[j]);
        prob05[j] = inv_logit(eta05[j]);
        }

    }

"""

### Settings for running STAN

In [19]:
iterations    = 8000
chains        = 2                           # HMC chains
warmup        = 2000                        # How many of the first iterations we'll ignore - burnin
jobs          = -1                          # Run code in parallel -- see pystan documentation
seed          = 1

In [20]:
control = {}
# control['max_treedepth'] = 20
control['adapt_delta'] = 0.85

### The fit:

In [22]:
start = time.time()

fit = pystan.stan(model_code=stan_code, data=regression_data, seed=seed, iter=iterations, chains=chains, 
                  warmup=warmup, n_jobs=jobs, control=control)

end = time.time()

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_902d543734551fafeca59854d8a03fd0 NOW.
/home/mlldantas/anaconda3/envs/py2k7/lib/python2.7/site-packages/Cython/Compiler/Main.py:367: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /tmp/tmpW5eYgE/stanfit4anon_model_902d543734551fafeca59854d8a03fd0_7675752649170245185.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [23]:
print ("--------------------------------------------------")
print ("CPU process time: %.2f [min]" % float((end-start)/60))
print ("--------------------------------------------------")

--------------------------------------------------
CPU process time: 9.41 [min]
--------------------------------------------------


### Extracting fit properties

In [24]:
summary = pystan.stansummary(fit=fit, digits_summary=3)

In [25]:
summary_arr = np.array(summary.split('\n'))

In [26]:
new_output = summary_arr[5:-6,]

In [27]:
# count = 0
# for i in range(new_output.size):
#     row = np.array(new_output[i].split())
#     if row.size ==11:
#         continue
#     else:
#         print "new_output[%d] = '%s' " % (i, str(new_output[i]))
#         count+=1
# print count

new_output[15] = 'beta[1,4]    -0.15   0.002   0.24  -0.65 -0.294 -0.137-4.17e-4  0.312  13106    1.0' 
new_output[4549] = 'eta03[23]   -0.042    0.01  1.041 -2.187 -0.6532.45e-4  0.602  1.946   9954    1.0' 
new_output[4550] = 'eta03[24]   -0.043    0.01  1.041  -2.19 -0.6543.95e-4  0.602  1.945   9962    1.0' 
new_output[4551] = 'eta03[25]   -0.044    0.01  1.041 -2.191 -0.653-5.31e-4    0.6  1.943   9970    1.0' 
new_output[4554] = 'eta03[28]   -0.046    0.01  1.041 -2.193 -0.656-3.58e-5  0.598   1.94   9996    1.0' 
new_output[4566] = 'eta03[40]   -0.038    0.01  0.979 -2.067 -0.6118.07e-4  0.572  1.826   9852    1.0' 
new_output[4568] = 'eta03[42]   -0.039    0.01  0.979 -2.068 -0.6127.95e-4   0.57  1.824   9867    1.0' 
new_output[4569] = 'eta03[43]    -0.04    0.01  0.979 -2.067 -0.6122.41e-4  0.569  1.823   9874    1.0' 
new_output[4570] = 'eta03[44]    -0.04    0.01  0.979 -2.067 -0.612-1.0e-4  0.568  1.821   9882    1.0' 
new_output[4576] = 'eta03[50]   -0.044    0.01  0.979 

In [28]:
# new_output[15] = 'beta[1,4]    -0.15   0.002   0.24  -0.65 -0.294 -0.137 -4.17e-4  0.312  13106    1.0' 
# new_output[4549] = 'eta03[23]   -0.042    0.01  1.041 -2.187 -0.653 2.45e-4  0.602  1.946   9954    1.0' 
# new_output[4550] = 'eta03[24]   -0.043    0.01  1.041  -2.19 -0.654 3.95e-4  0.602  1.945   9962    1.0' 
# new_output[4551] = 'eta03[25]   -0.044    0.01  1.041 -2.191 -0.653 -5.31e-4    0.6  1.943   9970    1.0' 
# new_output[4554] = 'eta03[28]   -0.046    0.01  1.041 -2.193 -0.656 -3.58e-5  0.598   1.94   9996    1.0' 
# new_output[4566] = 'eta03[40]   -0.038    0.01  0.979 -2.067 -0.611 8.07e-4  0.572  1.826   9852    1.0' 
# new_output[4568] = 'eta03[42]   -0.039    0.01  0.979 -2.068 -0.612 7.95e-4   0.57  1.824   9867    1.0' 
# new_output[4569] = 'eta03[43]    -0.04    0.01  0.979 -2.067 -0.612 2.41e-4  0.569  1.823   9874    1.0' 
# new_output[4570] = 'eta03[44]    -0.04    0.01  0.979 -2.067 -0.612 -1.0e-4  0.568  1.821   9882    1.0' 
# new_output[4576] = 'eta03[50]   -0.044    0.01  0.979 -2.062 -0.619 -9.06e-4  0.563  1.825   9930    1.0' 
# new_output[4577] = 'eta03[51]   -0.045    0.01  0.979 -2.063 -0.619 -8.97e-4  0.562  1.824   9939    1.0' 
# new_output[4588] = 'eta03[62]   -0.038   0.009  0.918 -1.942 -0.574 -1.1e-4  0.535  1.707   9789    1.0' 
# new_output[4589] = 'eta03[63]   -0.038   0.009  0.918 -1.941 -0.574 -8.22e-5  0.534  1.704   9797    1.0' 
# new_output[4590] = 'eta03[64]   -0.039   0.009  0.918  -1.94 -0.574 -8.14e-4  0.534  1.702   9804    1.0' 
# new_output[6654] = 'eta04[328]  -0.011   0.004  0.447  -0.95 -0.271 -1.5e-4  0.255   0.89  12833    1.0' 
# new_output[6655] = 'eta04[329]  -0.011   0.004  0.448 -0.952 -0.272 -8.37e-4  0.253   0.89  12850    1.0' 
# new_output[6797] = 'eta04[471]   -0.16   0.002  0.255 -0.705 -0.314 -0.147 5.21e-4  0.331  12442    1.0' 
# new_output[6799] = 'eta04[473]  -0.162   0.002  0.257 -0.706 -0.316 -0.149 2.68e-4  0.332  12381    1.0' 
# new_output[6800] = 'eta04[474]  -0.162   0.002  0.258  -0.71 -0.318 -0.149 1.43e-4  0.332  12351    1.0' 
# new_output[6801] = 'eta04[475]  -0.163   0.002  0.259 -0.715 -0.319  -0.15 -8.54e-6  0.333  12320    1.0' 
# new_output[6802] = 'eta04[476]  -0.164   0.002   0.26 -0.719  -0.32 -0.151 2.92e-4  0.336  12291    1.0' 
# new_output[6803] = 'eta04[477]  -0.165   0.002  0.261 -0.723 -0.322 -0.152 4.04e-4  0.336  12261    1.0' 
# new_output[6804] = 'eta04[478]  -0.166   0.002  0.263 -0.726 -0.323 -0.152 2.75e-4  0.338  12231    1.0' 
# new_output[6805] = 'eta04[479]  -0.167   0.002  0.264 -0.729 -0.325 -0.154 -4.16e-4  0.339  12202    1.0' 
# new_output[6806] = 'eta04[480]  -0.168   0.002  0.265 -0.735 -0.327 -0.154 -8.02e-4   0.34  12173    1.0' 
# new_output[8638] = 'eta05[512]    -0.1   0.001  0.153  -0.41 -0.199 -0.097 8.04e-4  0.194  13136    1.0' 
# new_output[8639] = 'eta05[513]  -0.101   0.001  0.153  -0.41   -0.2 -0.098 7.67e-4  0.194  13131    1.0' 
# new_output[8640] = 'eta05[514]  -0.101   0.001  0.152 -0.411 -0.201 -0.098 1.66e-4  0.193  13125    1.0' 
# new_output[8641] = 'eta05[515]  -0.102   0.001  0.152 -0.411 -0.201 -0.099 -6.62e-4  0.193  13120    1.0'

In [29]:
count = 0
for i in range(new_output.size):
    row = np.array(new_output[i].split())
    if row.size ==11:
        continue
    else:
        print "new_output[%d] = '%s' " % (i, str(new_output[i]))
        count+=1
print count

0


In [30]:
header_fit = summary_arr[4].split()
print header_fit

[u'mean', u'se_mean', u'sd', u'2.5%', u'25%', u'50%', u'75%', u'97.5%', u'n_eff', u'Rhat']


In [31]:
header_addendum = 'parameter'
header_fit = [header_addendum] + header_fit
print header_fit

['parameter', u'mean', u'se_mean', u'sd', u'2.5%', u'25%', u'50%', u'75%', u'97.5%', u'n_eff', u'Rhat']


In [32]:
cute_output = list(np.zeros(len(header_fit)))
for i in range(new_output.size):
    if len(new_output[i].split())!=11: # the length of the list must be 11, in which case we connect them directly
        print "there is a problem!"
    else:
        new_output_temp = np.array(new_output[i].split()).reshape(1,11)
        cute_output     = np.vstack((cute_output, new_output_temp))
cute_output = cute_output[1:,:]               # removing the zeroes in the beggining 

# Extracting and saving *ONLY* what really matters for the analysis

In [66]:
recovered_mass  = un_standardize(stdized_par=plot_x2, mean=mean_mass, std=std_mass)
regression_pars = np.column_stack((plot_x1, recovered_mass))
rp              = regression_pars
rp_cols         = np.vstack((rp, rp, rp, rp, rp))       # for 5 whan classes we must stack these 5x

In [79]:
print np.unique(plot_x2).size

30


In [72]:
parameters = cute_output[:,0].astype(str)
pnew_idxs  = []
for i in range(parameters.size):
    if parameters[i][0:4]=='prob':
        pnew_idxs.append(i)
    else:
        continue

model_results    = np.column_stack((cute_output[pnew_idxs,:], rp_cols))
model_results_df = pd.DataFrame(model_results)
model_results_df.columns = header_fit + ['Z'] + ['LOG_STELLAR_MASS']
model_results_df.to_csv('./Results/fit_results_sharedprior.csv', sep=',', header=True, index=False)

(4500, 11)
(4500, 2)


In [82]:
pd.DataFrame(model_results[30:70,:])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,prob01[31],0.35,0.003,0.158,0.078,0.225,0.344,0.467,0.664,3638,1.001,0.06794,9.796691931034484
1,prob01[32],0.35,0.003,0.158,0.078,0.225,0.344,0.467,0.663,3642,1.001,0.07760758620689655,9.796691931034484
2,prob01[33],0.35,0.003,0.158,0.078,0.225,0.343,0.467,0.663,3646,1.001,0.0872751724137931,9.796691931034484
3,prob01[34],0.35,0.003,0.158,0.078,0.225,0.343,0.466,0.663,3650,1.001,0.09694275862068966,9.796691931034484
4,prob01[35],0.35,0.003,0.158,0.078,0.225,0.343,0.466,0.663,3654,1.001,0.1066103448275862,9.796691931034484
5,prob01[36],0.35,0.003,0.158,0.078,0.225,0.343,0.466,0.662,3658,1.001,0.11627793103448275,9.796691931034484
6,prob01[37],0.35,0.003,0.158,0.078,0.225,0.343,0.466,0.662,3663,1.001,0.1259455172413793,9.796691931034484
7,prob01[38],0.35,0.003,0.158,0.078,0.224,0.343,0.466,0.662,3667,1.001,0.13561310344827587,9.796691931034484
8,prob01[39],0.35,0.003,0.158,0.078,0.224,0.343,0.466,0.662,3672,1.001,0.1452806896551724,9.796691931034484
9,prob01[40],0.349,0.003,0.158,0.078,0.224,0.343,0.466,0.663,3676,1.001,0.15494827586206894,9.796691931034484


In [ ]:
posteriors_temp = list(fit.extract(u'beta').items()[0])

In [ ]:
posteriors = np.array(posteriors_temp[1])

In [ ]:
print posteriors.shape
print posteriors[0,:,0] # row -- b0,b1,b2,b3,b4  --- betas
print posteriors[0,:,:] # col -- w0,w1,w2,w3,w4  --- emission-line classification (WHAN)

In [ ]:
blocks            = posteriors[:,0,0].size  # number of blocks of betas we have
number_of_classes = n_class
betas_size        = regression_data['K']

# Mega-loop to extract and plot the posteriors

In [ ]:
betas_dictionary = {}
plots = {}
colour='black'

In [ ]:
for each_class in range(number_of_classes):
    for each_beta in range(betas_size):
        print each_class, each_beta
        betas_dictionary["beta{0}{1}".format(each_class, each_beta)] = posteriors[:, each_beta, each_class]
        print np.array(betas_dictionary.values()).size
#         print np.array(betas_dictionary.values()).shape
#         plot_position = len(betas_dictionary.keys())
#         print each_beta, each_class, betas_dictionary.keys()[0]
# #         for p in range(plot_position):
#             plt.subplot(each_beta, each_class, p)
#             sns.kdeplot(posteriors[:, each_beta, each_class], shade=True, c=colour)
#             plt.xlabel(r"$\beta_{{0}{1}}$".format(each_beta, each_class), fontsize=10)
#             plt.tick_params('both', labelsize='10')
           
            
#             plot = my_plots(x=posteriors[:, each_beta, each_class], row=each_beta, col=each_class, position=p)
#             plt.show()   